# Group: CopyPaste
#### Dataset used: Dataset_Hospital_Vists.csv, test.csv
#### Participants: Ligia, Gina, Raza

### Problem
#### Many patients miss their scheduled appointments.

### Goal 
#### Develop a machine learning model that predicts if a patient will miss a future appointment.

# Still need to check

#### Duplicates in appointment id: follow ups?
#### Create heatmap of correlation
#### How does the time affect the outcome?
#### Is appointment date > scheduled data?
#### Distribution of the target variable.

# Thoughts on pre processing:
#### Change the no show to show to make it more intuitive.

# Preprocessing

### Missing Data:

#### Age, community, social welfare + disease info has missing data
